In [9]:
#imports
import os
import nltk
import re, string
import treetaggerwrapper
import json

In [10]:
#before uploading files from the dataset, files which weight 0 bytes and duplicates were removed
#read each file in the dataset and put it to a list
all_texts = []
names = os.listdir("/Users/valeriia/archive")
for name in names:
    if name.endswith(".txt"):
        text = open("/Users/valeriia/archive/"+name).read()
        all_texts.append(text)

In [13]:
#function for removing special characters before tokenization
#since "&" and "%" stand for actual words which are pronounced, they are kept but transformed into words
#in order not to lose newlines "\n" during tokenization, each newline is turned into "###"
def remove_special_characters(sentence):
    sentence = sentence.strip()
    characters = r"[.|,|:|!|?|*|(|)|~|-|–|„|“|●|#|/|\"|\'|»|«]"
    sentence = re.sub(characters, r"", sentence)
    repeat = r"[x+0-9]"
    sentence = re.sub(repeat, r"", sentence)
    if "&" in sentence:
        sentence = re.sub("&", r" and ", sentence)
    if "%" in sentence:
        sentence = re.sub("%", r" Prozent", sentence)
    if "-" in sentence:
        sentence = re.sub("-", r" ", sentence)
    if "\n" in sentence:
        sentence = re.sub("\n", r" ### ", sentence) 
    return sentence

all_texts_cleaned = [remove_special_characters(sentence) for sentence in all_texts]

In [15]:
#if a word starts with a lower or a capital letter may be crucial in German
#every single word in the set is tagged with POS
#if POS of a word is anything but noun, it is converted to lower case
tagger = treetaggerwrapper.TreeTagger(TAGLANG='de', TAGDIR="/Users/valeriia")
all_texts_cleaned2 = []
for text in all_texts_cleaned:
    text_tokens = []
    text_pos = tagger.tag_text(text)
    text_pos2 = treetaggerwrapper.make_tags(text_pos)
    for word in text_pos2:
        try:
            if word.pos == "NN" or word.pos == "NE":
                text_tokens.append(word.word)
            else:
                text_tokens.append((word.word).lower())
        except:
            continue
    text_cleaned = " ".join(text_tokens)
    all_texts_cleaned2.append(text_cleaned)

In [16]:
#function for putting newlines "/n" back so that they can be used for marking borders between lines and refrains
all_texts_cleaned3 = []
def spaces(text):
    if "###" in text:
        text = re.sub(" ### ", r"\n", text)
        text = re.sub("### ", r"\n", text)
        text = re.sub("###", r"\n", text)
    all_texts_cleaned3.append(text)
    
for text in all_texts_cleaned2:
    spaces(text)

In [6]:
#save texts to a json file - each song is a separate string
jsonString = json.dumps(all_texts_cleaned3, indent=4, ensure_ascii=False)
jsonFile = open("all_texts_cleaned.json", "w", encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()

In [7]:
#split the set into refrains
refrains = []
for text in all_texts_cleaned3:
    refrain = ""
    lines = text.splitlines( )
    for line in lines:
        refrain += line+"/n"
        if line == "" and refrain != "/n":
            if refrain.startswith("/n"):
                refrain = refrain[2:]
            refrains.append(refrain)
            refrain = ""

In [8]:
#save texts to a json file - each refrain is a separate string
jsonString = json.dumps(refrains, indent=4, ensure_ascii=False)
jsonFile = open("refrains_cleaned.json", "w", encoding='utf-8')
jsonFile.write(jsonString)
jsonFile.close()